In [1]:
import nltk
import keras
import numpy as np
import sklearn

from sklearn.preprocessing import OneHotEncoder
from keras import layers
from keras import models
from keras.models import load_model
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

import tensorflow as tf

Using TensorFlow backend.


In [3]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()

#split the data into train/test
train, test = sklearn.model_selection.train_test_split(tagged_sentences, test_size=0.2, random_state=1);

#removes -NONE- tags
def cleanup(dat):
    l =[]
    
    for di in dat:
        l2= []
        count += 1
        for x,y in di:
            
            if y == '-NONE-':
                continue
            if y == 'SYM':
                continue
            
            l2.append((x,y))
        l.append(l2)
    
    return l
        
train = cleanup(train)
test = cleanup(test)

3131
783


In [18]:
#splits the data into X and y. Where X contains the sentences and y the corresponding tags
def transform(sentence):
    
    X, y = [],[];
    for x in sentence:
        for i in range(len(x)):
            
            X.append(x[i][0]);
            y.append(x[i][1]);


    return X, y;

#creates a dictionarie: number to tag/word
def tokenize(data_text):
    
    dic = dict()
    i = 0
    for x in data_text:
        if x in dic:
            continue
        dic[x] = i
        i+=1

    return dic


def to_hot(dat, dic):
    l=[]
    
    for tu in dat:
        for x,y in tu:
           
            l.append([dic[x]])
    
    return l

def myEncoder(ar,dic):
    l= []
    for y in ar:
        l.append(dic[y])
        
    return l
    

In [24]:
#split the data into x and y
X_train, y_train = transform(train)
X_test, y_test = transform(test)



word_index = tokenize(X_train + X_test)
tag_index = tokenize(y_train + y_test)
number_to_word = {v: k for k, v in word_index.items()}
number_to_tag = {v: k for k, v in tag_index.items()}


#encode X and y data
y_train = myEncoder(y_train, tag_index)
y_test = myEncoder(y_test, tag_index)
    
X_train = to_hot(train, word_index)
X_test = to_hot(test, word_index)


onehotencoder = OneHotEncoder(handle_unknown='ignore')
onehotencoder.fit(X_train + X_test)


X_train = onehotencoder.transform(X_train).toarray()
X_test = onehotencoder.transform(X_test).toarray()


y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)


In [9]:


def model(input_dim, hidden_neurons, output_dim):
    model = models.Sequential([
        Dense(hidden_neurons, input_dim=input_dim, activation='relu'),
        Dropout(0.2),
        Dense(hidden_neurons, activation='relu'),
        Dropout(0.2),
        Dense(output_dim, activation='sigmoid')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

mod = model(X_train.shape[1], 128, y_train.shape[1])


mod.fit(X_train, y_train, epochs=2, validation_split =0.2, batch_size = 32)


Train on 60281 samples, validate on 15071 samples
Epoch 1/2
60281/60281 [==============================] - 149s 2ms/step - loss: 1.0966 - acc: 0.6884 - val_loss: 0.4312 - val_acc: 0.8770
Epoch 2/2
60281/60281 [==============================] - 143s 2ms/step - loss: 0.2479 - acc: 0.9328 - val_loss: 0.3721 - val_acc: 0.8853


In [25]:
score = mod.evaluate(X_test, y_test)
print("The score is:",score[1])

18731/18731 [==============================] - 7s 392us/step
The score is: 0.8791842400267148


In [99]:
#mod.save('Hot_NLP_model.h5')
#mod = load_model('Hot_NLP_model.h5')

In [26]:

def translate(words, dic):
    sent = words.split()
    l = []
    for x in sent:
        l.append([dic[x]])

    p = onehotencoder.transform(l).toarray()
    
    encoded_tags = mod.predict_classes(p)
    
    print(encoded_tags)
    
    l=[]
    for e in encoded_tags:
        l.append(number_to_tag[e])
            

    return l

In [37]:
translate("which one would you like", word_index)

[27  7 12 11  6]


['WDT', 'CD', 'MD', 'PRP', 'IN']